In [ ]:
require 'watir'
require 'nokogiri'
require 'dotenv/load'

root_path = ENV['URL'] + 'categorias/'

In [ ]:
# browser = Watir::Browser.new :chrome, headless: true #headless chrome
browser = Watir::Browser.new :chrome

'Start browser'

In [ ]:
browser.goto root_path

In [ ]:
document = Nokogiri::HTML(browser.html)
'Parse html'

In [ ]:
categories = document.css("a[class='tags']")

'Fetch all categories'

In [ ]:
url = ENV['URL']

categories = categories.map { |c| 
    path = c.attr('href')
    { 
      category: c.text.delete!("\n"),
      path: url + path 
    }
  }

In [ ]:
salade_category = categories.select { |c| c[:category].include? 'Salada' }.first

In [ ]:
browser.goto salade_category[:path]

In [ ]:
pages = Nokogiri::HTML(browser.html)

'Parse html'

In [ ]:
last_page = pages.css('.row a').map { |a| 
    a.text
  }.reject { |a| 
    a.include? '>' or a.include? '<' 
  }.last

In [ ]:
recipies = (1..2).map { |index| #just to do a small fetch
#recipies = (1..last_page.to_i).map { |index| # run over all pages, commonly around 2k

    puts "Page index #{index}"
    browser.goto salade_category[:path] + "?page=#{index}"
    recipies = browser.elements(class: ['box-hover', 'box-big-item']).map { |e|
      e.href
    }.map { |r_url| 
      browser.goto r_url
      recipie = Nokogiri::HTML(browser.html)
      ingredients = recipie.at_css("div[id='info-user']").css('li').map { |l| 
        l.text
      }
      recipe_name = recipie.at_css("div[class='recipe-title']").css('h1').text.delete!("\n")
      { 
        title: recipe_name,
        ingredients: ingredients,
        url: browser.url
      }
    }
    recipies
  }


In [ ]:
r = recipies.flatten.select { |r| 
    r[:ingredients].any? { |i| 
        i.include? 'macarrão'
      }
  }

r.each { |r| 
    puts r[:title]
  }

In [ ]:
File.open('dump.csv', 'w') { |file|
  file.puts('title, ingredients, url') # file.puts = new line
  recipies.flatten.map { |r| 
    puts "#{r[:title]},#{r[:ingredients]},#{r[:url]}"
    file.puts("#{r[:title]},#{r[:ingredients]},#{r[:url]}")
  }
}
